In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pp
import analyse as ana
import MDAnalysis as mda
import pandas as pd
from MDAnalysis.analysis.base import AnalysisBase
from matplotlib.gridspec import GridSpec
from IPython.display import display, Markdown, Latex

%matplotlib widget

In [ ]:
from matplotlib.font_manager import FontProperties

textwidth = 6.50128 # latex textwidth

font0 = FontProperties()

font = font0.copy()
font.set_family('sans-serif')
font.set_style('normal')
font.set_weight('bold')
font.set_size(16)

Title = font.copy()
Title.set_size(18)

font2 = Title.copy()
font2.set_size(12)

font3 = font.copy()
font3.set_size(12)
font3.set_weight('normal')

mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['font.size'] = 12
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['axes.labelweight'] = 'bold'
mpl.rcParams['font.family'] = 'sans-serif'

mpl.rcParams['figure.figsize'] = (textwidth, 3)

In [ ]:
# Color-blind friendly pallette
CB_color_cycle = ['k', '#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=CB_color_cycle)

styles = {
    'NMR' : dict(linestyle='-'),
    'CTPOL' : dict(linestyle='--'),
    'opt-CTPOL' : dict(linestyle='-.')    
}

resabbrev = dict(
    ALA = 'A',
    ARG = 'R',
    ASN = 'N',
    ASP = 'D',
    CYS = 'C',
    GLN = 'Q',
    GLU = 'E',
    GLY = 'G',
    HIS = 'H',
    ILE = 'I',
    LEU = 'L',
    LYS = 'K',
    PHE = 'F',
    PRO = 'P',
    SER = 'S',
    THR = 'T',
    TRP = 'W',
    TYR = 'Y',
    VAL = 'V'
)

ref = 'trajectories/input.pdb'
parsets = [
    'CTPOL', 'opt-CTPOL'
]

unis = dict(NMR=ana.Universe(ref, ref))
for par in parsets:
    dcd = f"trajectories/MD-1ZNF/{par}/centered_output.dcd"
    unis[par] = ana.Universe('output.pdb', dcd)

names = {
    'protein' : 'protein',
    'nitrogens' : 'protein and name N*',
    'C_alpha' : 'backbone and name CA',
    'oxygens' : 'protein and name O*',
    #'water_O' : 'resname HOH and name O*',
    'backbone_O' : 'backbone and name O*',
    'acid_O' : '(not backbone) and (resname ASP or resname GLU) and name O*' 
}

In [ ]:
class DistanceAnalysis(AnalysisBase):
    r"""
    
    """
    def __init__(self, uni, seltxt, low, high, noH=True,
                 **kwargs):
        super(DistanceAnalysis, self).__init__(
            uni.trajectory, **kwargs)
        self.seltxt = seltxt
        self.low = low
        self.high = high
        self.u = uni
        self.noH = noH
        
    def _prepare(self):
        
        self.sltxt_low = f"(around {self.low} ({self.seltxt}))"\
                    + self.noH*" and not name H*"
        self.sltxt_high = f"(around {self.high} ({self.seltxt}))"\
                    + self.noH*" and not name H*"
        
        sel_low = self.u.select_atoms(self.sltxt_low)
        sel_high = self.u.select_atoms(self.sltxt_high)
        self.results.atoms = sel_high - sel_low
        
        self.results.count = {
            atom : 0 for atom in self.results.atoms
        }
        self.frames_calculated = 0
        
    def _single_frame(self):
        sel_low = self.u.select_atoms(self.sltxt_low)
        sel_high = self.u.select_atoms(self.sltxt_high)
        new_sel = sel_high - sel_low
        
        #self.results.atoms = (new_sel
        #- self.results.atoms) + self.results.atoms
        
        for atom in new_sel:
            if atom not in self.results.count:
                self.results.count[atom] = 1
            else:
                self.results.count[atom] += 1
          
        self.frames_calculated += 1
    def _conclude(self):
        self.results.atoms, self.results.count \
        = list(zip(*self.results.count.items()))
        self.results.count = np.array(self.results.count)
        self.results.count = self.results.count / self.frames_calculated

In [ ]:
        
def get_table(DAs):

    for m, (par, DA) in enumerate(DAs.items()):
        count = DA.results.count
        atoms = DA.results.atoms

        atom_labels = []
        for n, atom in enumerate(atoms):
            name = atom.name
            resname = atom.resname
            resid = atom.resid + 1*(par == 'NMR')
            
            if resname in resabbrev:
                res = resabbrev[resname]
            else:
                res = resname
            
            label = f"{name}:{res}{resid}"
            atom_labels.append(label)
        
        new_df = pd.DataFrame(
            {f"{par}" : count}, index=atom_labels
        )
            
        if m == 0:
            df = new_df
        else:
            df = df.join(new_df, how='outer')
                
    return df.fillna(0)
        

# RDF Plot

In [ ]:
fig = pp.figure()
gs = GridSpec(3, 1, hspace=0.5, bottom=0.0)

ax = fig.add_subplot(gs[:-1, :])
ax2 = fig.add_subplot(gs[-1:, :], sharex=ax)

#fig, axes = pp.subplots(1,2)#2, sharex=True)
for par, uni in unis.items():
    g1 = uni.select_atoms('name ZN')
    g2 = uni.select_atoms('protein and not name H*')
    rdf = ana.InterRDF(g1, g2, nbins=600, range=(1,8))
    rdf.run()

    ax.plot(rdf.results.bins, rdf.results.rdf, label=par,  linewidth=2)
    ax.set_xlim(1.5, 4)
    ax.set_ylim(0, None)

ax.set_xlabel(r"$r (\AA)$")
ax.set_ylabel("RDF")
ax.set_xticklabels([])
ax.legend()
#ax.set_title("RDF of non-hydrogen protein atoms")

### Manual peak determination from Plot and visualize

In [ ]:
Input = {}
Input[1] = {
    'NMR' : [1.8, 2.15],
    'CTPOL' : [1.7, 2.3],
    'opt-CTPOL' : [1.5, 1.89]
}
Input[2] = {
    'NMR' : [2.15, 2.5],
    'CTPOL' : [1.7, 2.3],
    'opt-CTPOL' : [1.89, 2.3]
}
Input[3] = {
    'NMR' : [2.5, 3.70],
    'CTPOL' : [2.3, 3.75],
    'opt-CTPOL' : [2.3, 3.5]
}

In [ ]:
ax2.clear()
ax2.axis('off')

for n, par in enumerate("NMR CTPOL opt-CTPOL".split()):
    color = CB_color_cycle[n]
    for m in Input:
        xcoords = np.array(Input[m][par])
        x = xcoords.mean()
        y = -2*n
        
        text = f"Peak {m}"
        if m == 2 and par == 'CTPOL':
            text = "Peak 1 & 2"
        t = ax2.annotate(text, [x, y] , fontsize=8, ha='center', va='center', color=color, weight='bold')
        t.set_bbox(dict(facecolor='w', alpha=1, edgecolor=color))
        
        ax2.plot(xcoords, [y, y], color=color, marker="|", markersize=10)
        
ylims = np.array([-2.5*n, + 1])
ax2.set_ylim(ylims)
ax.xaxis.set_label_coords(0.5, -0.05)
fig

In [ ]:
fig.savefig('Images/rdf_all.pdf', dpi=300)

In [ ]:
Data = {}

display(Markdown(f"# Determining Peak composition"))
for p, inp in Input.items():
    DAs = {}
    display(Markdown(f"## Peak no. {p}"))
    
    for par, (low, high) in inp.items():
        display(Markdown(f"### {par}"))
        print(f"low = {low}, high = {high}")
        DA = DistanceAnalysis(
            unis[par], 
            'name ZN',
            low, high
        )
        DA.run()
        DAs[par] = DA
        
    Data[p] = DAs
#get_table(DAs)

In [ ]:
def group_hoh(df):
    def mapping_func(index):
        if 'HOH' in index:
            return 'HOH'
        else:
            return index
    return df.groupby(mapping_func).mean()

def group_rare(df, cutoff=0.01, exceptions=None):
    if exceptions is None:
        exceptions = []
    def mapping_func(index):
        if index in exceptions:
            return index
        elif (df.loc[index] < cutoff).all():
            return 'Others'
        else:
            return index
    return df.groupby(mapping_func).mean()

def group_similar(df):
    def mapping_func(index):
        if "," in index: 
            return index
        numberless = index.rstrip("0123456789")
        all_numbers = []
        for ind in df.index:
            if ind.startswith(numberless):
                all_numbers.append(ind.replace(numberless, ""))
        new_index = numberless + ",".join(list(set(all_numbers)))
        return new_index
    return df.groupby(mapping_func).mean()

#fig, axes = pp.subplots(1,len(Data), sharey=True)
fig = pp.figure()
#fig.suptitle("Composition of the peaks")

gs = GridSpec(1, 4, wspace=0.01, bottom=0.35)

ax1 = fig.add_subplot(gs[:, :1])
ax2 = fig.add_subplot(gs[:, 1:2], sharey=ax1)
ax3 = fig.add_subplot(gs[:, 2:], sharey=ax1)


axes = fig.axes
for n, p in enumerate(Data):
    df = group_similar(
           group_rare(
             group_hoh(
                get_table(Data[p])
            ), cutoff=0.02, exceptions = ['HOH']
        )
    )
    df = df.sort_values(['NMR', 'opt-CTPOL', 'CTPOL'], ascending=False)
    #display(Markdown(f'### Peak {p}'))
    idx = df.index.tolist()
    idx.pop(idx.index('Others'))
    idx = idx + ['Others']
    df = df.loc[idx]
    #display(df)
    
    #'''
    ax = axes[n]
    df.plot.bar(
        ax=ax, 
        color=CB_color_cycle, 
        legend=False,
        width=0.9
        
    )
    '''
    ax = axes[n]
    for col in df.columns:
        ax.step(
            #ax=ax, 
            #color=['k', CB_color_cycle[0], CB_color_cycle[1]],
            df.index, df[col],
            label=col,
            linewidth=2.5,
            alpha = 1,
            **styles[col]
        )    
    ax.set_xticks(ax.get_xticks())
    
    ax.set_xticklabels(df.index, rotation='vertical')
    #'''
    
    ax.set_yscale('log')
    ax.set_title(f"Peak {p}")
    for par in Data[p]:
        print(f"{par} : {Data[p][par].low} | {Data[p][par].high}")
        
handles, labels = ax.get_legend_handles_labels()
#fig.legend(handles, labels, 'upper right' )     

In [ ]:
fig.savefig('Images/Peak_breakdown_log.pdf')

# Todos

* DFT Distance distributions?
* Try to explain shorter distances due to parametrization process.
* Label the peaks
* Can we get DFT data in the plot?
    - Perhaps just individual atom distances and compare?